<a href="https://colab.research.google.com/github/ntgr8r/froala-design-blocks/blob/master/PyTorch_Tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from os import path
from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())

accelerator = 'cu80' if path.exists('/opt/bin/nvidia-smi') else 'cpu'
!pip install -q http://download.pytorch.org/whl/{accelerator}/torch-1.0.0-{platform}-linux_x86_64.whl torchvisio

In [0]:
# Plan
# Install PyTorch
# Introduction to Tensors
# GPU and You
# Datasets and DataLoaders
# Your first model
  # Loading in data
  # Setting an optimizer
  # Training and testing
  # Saving a checkpoint
  # Saving a model
  # Using Tensorboard

In [0]:
import torch
import pandas as pd
import numpy as np
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils

In [0]:
# This is a tensor row then column
x = torch.rand(4,4)
print("My first PyTorch tensor: ", x)
print("Tensor dimension: ", x.shape)

# In math + physics tensors have more complicated definitions, but for
# our purposes they're just numpy arrays that you can execute operations on
# with a GPU

My first PyTorch tensor:  tensor([[0.6634, 0.9268, 0.9097, 0.2191],
        [0.8996, 0.9523, 0.9412, 0.1074],
        [0.0630, 0.8249, 0.9781, 0.2342],
        [0.1591, 0.4465, 0.0053, 0.1909]])
Tensor dimension:  torch.Size([4, 4])


In [0]:
y = torch.rand(4,4)
# You can perform all the operations you'd expect on tensors
z = y.add(x)
print(z)

# Any mutation that mutates a tensor in place is post-fixed with a _
# for xample x.copy_(y), x.t_() will change x
y.add_(x)
print(y)


tensor([[1.1536, 1.0480, 1.1224, 0.6295],
        [1.3789, 0.9750, 1.4775, 0.1956],
        [0.2499, 1.1826, 1.3731, 0.5702],
        [0.5278, 1.2348, 0.1687, 0.8151]])
tensor([[1.1536, 1.0480, 1.1224, 0.6295],
        [1.3789, 0.9750, 1.4775, 0.1956],
        [0.2499, 1.1826, 1.3731, 0.5702],
        [0.5278, 1.2348, 0.1687, 0.8151]])


In [0]:
import numpy as np

# we can convert pytorch tensors to numpy arrays
x = torch.randn(5)
b = np.random.rand(5)
print(type(x))
print(type(b))
print(x)

y = x.numpy()
print(type(y))
print(y)

# y still points to x
x.add_(1)
print(x)
print(y)

<class 'torch.Tensor'>
<class 'numpy.ndarray'>
tensor([-0.6417,  0.3274,  0.0461, -0.6507,  0.5922])
<class 'numpy.ndarray'>
[-0.64170617  0.3274402   0.04614983 -0.65072805  0.59221715]
tensor([0.3583, 1.3274, 1.0461, 0.3493, 1.5922])
[0.35829383 1.3274403  1.0461498  0.34927195 1.5922172 ]


In [0]:
# torch.FloatTensor which is a 32 float is the default type
# Numpy default dtype is float64 while pytorch is float32

# Can assign tensors to dtype and to devices

if torch.cuda.is_available():
  device = torch.device('cuda')
  y = torch.ones_like(x, device=device)
  x = x.to(device)
  z = x + y
  print(z)
  print(z.to('cpu'))
  print(type(z))

tensor([1.8138, 1.4370, 0.6378, 2.1627, 3.6024], device='cuda:0')
tensor([1.8138, 1.4370, 0.6378, 2.1627, 3.6024])
<class 'torch.Tensor'>


In [0]:
# Make a dataset object
class CityData(Dataset):
  def __init__(self, csv_file, transform=None):
    self.city_data = pd.read_csv(csv_file)
    self.transform = transform
   
  def __len__(self):
    return len(self.city_data)
  
  def __getitem__(self, idx):
    return self.city_data.iloc[[idx]].values
 

In [0]:
example_data = CityData("https://download.mlcc.google.com/mledu-datasets/california_housing_train.csv")

dataloader = DataLoader(example_data, batch_size=4, shuffle=True)

print(len(dataloader.dataset))
for i_batch, sampled_batch in enumerate(dataloader):
  print(i_batch)
  print(sampled_batch)
  break

17000
0
tensor([[[-1.1915e+02,  3.4250e+01,  3.6000e+01,  3.5110e+03,  6.6400e+02,
           2.9650e+03,  6.9500e+02,  4.0878e+00,  1.8680e+05]],

        [[-1.2244e+02,  3.7750e+01,  4.6000e+01,  1.5190e+03,  2.9100e+02,
           5.7300e+02,  2.8900e+02,  4.2667e+00,  3.3880e+05]],

        [[-1.1733e+02,  3.3230e+01,  1.5000e+01,  1.9050e+03,  4.1600e+02,
           1.2580e+03,  3.8800e+02,  3.3300e+00,  1.2790e+05]],

        [[-1.2282e+02,  3.8390e+01,  2.2000e+01,  1.2880e+03,  2.4300e+02,
           5.9300e+02,  2.2000e+02,  3.6250e+00,  2.3370e+05]]],
       dtype=torch.float64)


In [0]:
# Now going to build a CNN to classify images
# Why CNN
# Quick overview of CNN
# MNIST dataset

In [0]:
sample = datasets.MNIST('../data', train=True, download=True, transform=None)
print("label: ", sample[0][1])
sample[0][0]

label:  tensor(5)


In [0]:
# Convert images to tensors

def tensor_to_image(tensor):
  return transforms.ToPILImage()(tensor)

sample = datasets.MNIST('../data', train=True, download=True, transform=transforms.ToTensor())
tensor_to_image(sample[0][0])

In [0]:
# For training we're going to normalize images so pixels are in range
# [-1, 1]
sample = datasets.MNIST('../data', train=True, download=True,
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ]))
tensor_to_image(sample[0][0])

In [0]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [0]:
# Going to show two versions of the same basic network and training.
# The first is to focus on the building blocks. Heavily commented.
# The second shows you
# how to do some useful things, like clean organization, saving and loading
# and visualizing training.

class Network(nn.Module):
  def __init__(self):
    
    # Use super because if child using cooperative multiple inheritance
    # then it sorts out the method resolution order. Which of the superclasses
    # (parent classes) should __init__ be called on first
    super(Network, self).__init__()
    
    # Tensor size going in
    # (32, 1, 28, 28)
    
    # In channels, out channels, kernal size, stride
    self.conv1 = nn.Conv2d(1, 20, 5, 1)
    self.conv2 = nn.Conv2d(20, 50, 5, 1)
    
    # In features, out features
    self.fc1 = nn.Linear(4*4*50, 500)
    self.fc2 = nn.Linear(500, 10)
   
  def forward(self, x):
    # Tensor shape going in
    # (32, 1, 28, 28)
    x = F.relu(self.conv1(x))
    
    # (32, 20, 24, 24)
    # input, kernel size, stride
    x = F.max_pool2d(x, 2, 2)

    # (32, 20, 12, 12)
    x = F.relu(self.conv2(x))
    
    # (32, 50, 8, 8)
    x = F.max_pool2d(x, 2, 2)

    
    # (32, 50, 4, 4)
    # Flatten tensor
    x = x.view(-1, 4*4*50)
    
    # (32, 800)
    x = F.relu(self.fc1(x))

    # (32, 500)
    x = self.fc2(x)

    # (32, 10)
    # Why log softmax? Can be more numerically stable
    # Subtraction instead of division
    return F.log_softmax(x, dim=1)
    

In [0]:
def train(model, device, train_loader, optimizer, epoch, log_interval):
  
  # By default models are initialized in train mode.
  # Some layers like BatchNorm and Dropout have different
  # behaviour in train and eval mode. Explicitly state 
  # what you're doing.
  model.train()
  for batch_idx, (data, target) in enumerate(train_loader):
    data, target = data.to(device), target.to(device)
    
    # backward() function see a few lines down accumulates
    # gradients so at the start of each minibatch you need
    # to wipe them out so that you don't accumulate across
    # mini batches
    optimizer.zero_grad()
    predictions = model(data)
    
    # negative log likelihood loss
    # loss = -log(y)
    # softmax assigns high confidence to a correct prediction
    # NLL will be small, when low confidence to the correct class
    # prediction NLL will be large
    loss = F.nll_loss(predictions, target)
    
    # Compute sum of gradients
    loss.backward()
    # Parameter update based on current gradients
    optimizer.step()
    
    if batch_idx % LOG_INTERVAL == 0:
      print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

      
def test(model, device, test_loader, epoch):
  # Means we don't use things like dropout and batchNorm
  model.eval()
  test_loss = 0
  correct = 0
  
  # Don't apply gradients to these tensors
  with torch.no_grad():
    for data, target in test_loader:
      data, target = data.to(device), target.to(device)
      predictions = model(data)
      
      # Sum output instead of returning the average of the output as above.
      # Use .item() to access one element tensor as a number. Over batch verusu
      # whole set
      test_loss += F.nll_loss(predictions, target, reduction='sum').item()
      
      # dim: dimension to reduce, and keepdim
      # dim = 1 means over all the predictions.
      # Keepdim = True means dim is retained
      prediction = predictions.argmax(dim=1, keepdim=True)
      
      # compute elementwise equality
      # view_as is view this tensor as the same as prediction - look into
      correct += prediction.eq(target.view_as(prediction)).sum().item()
      
    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))
    
      

In [0]:
USE_CUDA = True if torch.cuda.is_available() else False
DEVICE = torch.device("cuda" if USE_CUDA else "cpu")
SEED = 2
L_R = 0.01
EPOCHS = 4
BATCH_SIZE = 32
LOG_INTERVAL = 10
GLOBAL_ITER = 0

def main():
  
  # Set seed so results are reproducible
  torch.manual_seed(SEED)
  
  # num_workers = 0 => main process loading batches in
  # num_workers = N > 0 => each worker loads a single batch and
  # returns when done
  # pin_memory speeds up transfer of loading images on cpu and then
  # pushing them to GPU. Allocates samples in page-locked memory
  kwargs = {'num_workers': 1, 'pin_memory': True} if USE_CUDA else {}
  
  
  # swap color axis because
  # numpy image: H x W x C
  # torch image: C X H X W
  # To Tensor normalizes to [0,1]
  # we want to normalize to [-1, 1] so
  # perform another normalization with mean and std of MNIST
  train_loader = torch.utils.data.DataLoader(
      datasets.MNIST('../data', train=True, download=True,
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
      batch_size=BATCH_SIZE, shuffle=True, **kwargs)
  
  test_loader = torch.utils.data.DataLoader(
      datasets.MNIST('../data', train=False, transform=transforms.Compose([
                           transforms.ToTensor(),
                           transforms.Normalize((0.1307,), (0.3081,))
                       ])),
      batch_size=BATCH_SIZE, shuffle=True, **kwargs)


  model = Network().to(DEVICE)
  optimizer = optim.Adam(model.parameters(), lr=L_R)

  for epoch in range(1, EPOCHS + 1):
      train(model, DEVICE, train_loader, optimizer, epoch, LOG_INTERVAL)
      test(model, DEVICE, test_loader, epoch)



In [0]:
main()

In [0]:
# Add tensorboard
from tensorboardcolab import TensorBoardColab

Using TensorFlow backend.


In [0]:
USE_CUDA = True

MNIST_MEAN = 0.1307
MNIST_STD = 0.3081

class Config:
  
  def __init__(self, **kwargs):
    for key, value in kwargs.items():
      setattr(self, key, value)


model_config = Config(
    cuda = True if torch.cuda.is_available() else False,
    device = torch.device("cuda" if USE_CUDA else "cpu"),
    seed = 2,
    lr = 0.01,
    epochs = 4,
    save_model = False,
    batch_size = 32,
    log_interval = 100
)
    
class Trainer:
  
  def __init__(self, config):
    
    self.cuda = config.cuda
    self.device = config.device
    self.seed = config.seed
    self.lr = config.lr
    self.epochs = config.epochs
    self.save_model = config.save_model
    self.batch_size = config.batch_size
    self.log_interval = config.log_interval
    
    self.globaliter = 0
    self.tb = TensorBoardColab()
    
    torch.manual_seed(self.seed)

    kwargs = {'num_workers': 1, 'pin_memory': True} if self.cuda else {}

    self.train_loader = torch.utils.data.DataLoader(
        datasets.MNIST('../data', train=True, download=True,
                     transform=transforms.Compose([
                         transforms.ToTensor(),
                         transforms.Normalize((MNIST_MEAN,), (MNIST_STD,))
                     ])),
        batch_size=self.batch_size, shuffle=True, **kwargs)

    self.test_loader = torch.utils.data.DataLoader(
        datasets.MNIST('../data', train=False, transform=transforms.Compose([
                             transforms.ToTensor(),
                             transforms.Normalize((MNIST_MEAN,), (MNIST_STD,))
                         ])),
        batch_size=self.batch_size, shuffle=True, **kwargs)


    self.model = Network().to(self.device)
    self.optimizer = optim.Adam(self.model.parameters(), lr=self.lr)
      
  def train(self, epoch):
  
    self.model.train()
    for batch_idx, (data, target) in enumerate(self.train_loader):
      
      self.globaliter += 1
      data, target = data.to(self.device), target.to(self.device)

      self.optimizer.zero_grad()
      predictions = self.model(data)

      loss = F.nll_loss(predictions, target)
      loss.backward()
      self.optimizer.step()

      if batch_idx % self.log_interval == 0:
        print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                  epoch, batch_idx * len(data), len(self.train_loader.dataset),
                  100. * batch_idx / len(self.train_loader), loss.item()))
        self.tb.save_value('Train Loss', 'train_loss', self.globaliter, loss.item())
        torch.save(self.model.state_dict(), '{}_{}.pt'.format(self.globaliter, loss.item()))

  def test(self, epoch):
    self.model.eval()
    test_loss = 0
    correct = 0

    with torch.no_grad():
      for data, target in self.test_loader:
        data, target = data.to(self.device), target.to(self.device)
        predictions = self.model(data)

        test_loss += F.nll_loss(predictions, target, reduction='sum').item()
        prediction = predictions.argmax(dim=1, keepdim=True)
        correct += prediction.eq(target.view_as(prediction)).sum().item()

      test_loss /= len(self.test_loader.dataset)
      accuracy = 100. * correct / len(self.test_loader.dataset)

      print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
          test_loss, correct, len(self.test_loader.dataset), accuracy))

def main():
  
  trainer = Trainer(model_config)
  
  for epoch in range(1, trainer.epochs + 1):
      trainer.train(epoch)
      trainer.test(epoch)
      trainer.tb.flush_line('train_loss')

  if (trainer.save_model):
      torch.save(trainer.model.state_dict(),"mnist_cnn.pt")

  


In [0]:
main()

In [0]:
model = Network().to(DEVICE)
model.load_state_dict(torch.load("1001_0.012172117829322815.pt"))

In [0]:
!rm *.pt